In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 7.8 MB/s eta 0:00:00


In [ ]:
# https://docs.google.com/spreadsheets/d/1CAzsKw0GvI7la2iWbsViBGUTJILOjTYagwxvAvGtatg/edit?usp=sharing

import os
import json
import random
import re
from tqdm.auto import tqdm
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import numpy as np
import torch

import category_encoders as ce
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, accuracy_score
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
import optuna

In [ ]:
def set_seed(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    torch.use_deterministic_algorithms(False)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    print(f"Random seed set to {seed}")

In [ ]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

warnings.filterwarnings("ignore", category=UserWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {DEVICE}")

Random seed set to 42
Device: cpu


In [ ]:
INPUT_ROOT = "/content/drive/MyDrive/DataFlow2026/data"
WORK_DIR = "/content/drive/MyDrive/DataFlow2026/working"

ACADEMIC_CSV = f"{INPUT_ROOT}/academic_records.csv"
ADMISSION_CSV = f"{INPUT_ROOT}/admission.csv"
TEST_CSV = f"{INPUT_ROOT}/test.csv"

SUBMISSION_CSV = f"{WORK_DIR}/submission.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
admission_df = pd.read_csv(ADMISSION_CSV)
academic_df = pd.read_csv(ACADEMIC_CSV)

semester_order = [
    "HK1 2020-2021", "HK2 2020-2021",
    "HK1 2021-2022", "HK2 2021-2022",
    "HK1 2022-2023", "HK2 2022-2023",
    "HK1 2023-2024", "HK2 2023-2024",
    "HK1 2024-2025"
]
semester_mapping = {sem: i for i, sem in enumerate(semester_order)}

student_df = pd.merge(academic_df, admission_df, on="MA_SO_SV", how="left")
student_df["SEMESTER_INDEX"] = student_df["HOC_KY"].map(semester_mapping)

student_df = student_df.sort_values(
    by=["MA_SO_SV", "HOC_KY", "TC_DANGKY", "TC_HOANTHANH", "GPA"],
    ascending=[True, True, True, False, False]
)
student_df = student_df.drop_duplicates(subset=["MA_SO_SV", "HOC_KY"], keep="first")
student_df = student_df.sort_values(["MA_SO_SV", "SEMESTER_INDEX"]).reset_index(drop=True)

valid_semesters = ["HK2 2023-2024"]
train_df_raw = student_df[~student_df["HOC_KY"].isin(valid_semesters)].copy()
val_df_raw = student_df[student_df["HOC_KY"].isin(valid_semesters)].copy()

test_df_raw = pd.read_csv(TEST_CSV)
test_df_raw = pd.merge(test_df_raw, admission_df, on="MA_SO_SV", how="left")
test_df_raw["SEMESTER_INDEX"] = test_df_raw["HOC_KY"].map(semester_mapping)

print(f"Raw Train shape: {train_df_raw.shape}")
print(f"Raw Val shape: {val_df_raw.shape}")
print(f"Raw Test shape: {test_df_raw.shape}")

Raw Train shape: (90122, 12)
Raw Val shape: (15144, 12)
Raw Test shape: (16502, 9)


In [ ]:
EXAM_STATS_DETAILED = {
    2020: {
        "A00": {"mean": 21.4471, "std": 3.3425},
        "A01": {"mean": 20.0417, "std": 3.3084},
        "B00": {"mean": 20.3388, "std": 3.0812},
        "D01": {"mean": 18.1417, "std": 3.7811},
        "D07": {"mean": 20.0117, "std": 3.1333},
        "D24": {"mean": 22.8503, "std": 2.8891},
        "D29": {"mean": 22.3681, "std": 2.8403},
        "OTHER": {"mean": 19.5364, "std": 3.6661}
    },
    2021: {
        "A00": {"mean": 21.0262, "std": 3.1863},
        "A01": {"mean": 21.1029, "std": 3.4455},
        "B00": {"mean": 19.9892, "std": 3.0890},
        "D01": {"mean": 19.2666, "std": 4.1166},
        "D07": {"mean": 21.1338, "std": 3.2841},
        "D24": {"mean": 21.9896, "std": 3.0011},
        "D29": {"mean": 21.4177, "std": 3.0370},
        "OTHER": {"mean": 20.2071, "std": 3.7104}
    },
    2022: {
        "A00": {"mean": 21.0955, "std": 3.2378},
        "A01": {"mean": 20.2909, "std": 3.3396},
        "B00": {"mean": 19.4039, "std": 3.1555},
        "D01": {"mean": 18.4381, "std": 3.8846},
        "D07": {"mean": 20.2397, "std": 3.2064},
        "OTHER": {"mean": 19.5196, "std": 3.6548}
    },
    2023: {
        "A00": {"mean": 20.7745, "std": 3.0941},
        "A01": {"mean": 20.2743, "std": 3.3399},
        "B00": {"mean": 20.6047, "std": 2.7763},
        "D01": {"mean": 18.8891, "std": 3.8137},
        "D07": {"mean": 20.4216, "std": 3.0619},
        "D24": {"mean": 20.8364, "std": 3.3689},
        "D29": {"mean": 20.2285, "std": 3.5744},
        "OTHER": {"mean": 19.8594, "std": 3.4893}
    },
    2024: {
        "A00": {"mean": 20.9046, "std": 3.3804},
        "A01": {"mean": 20.4724, "std": 3.3509},
        "B00": {"mean": 20.5311, "std": 2.9818},
        "D01": {"mean": 19.4939, "std": 3.6232},
        "D07": {"mean": 20.4510, "std": 3.1120},
        "D24": {"mean": 21.8162, "std": 3.2930},
        "D29": {"mean": 20.9934, "std": 3.5598},
        "OTHER": {"mean": 20.1512, "std": 3.4281}
    },
}

In [ ]:
def get_features(input_df):
    df = input_df.copy()

    target_keys = set(zip(df["MA_SO_SV"], df["HOC_KY"]))
    student_filtered = student_df[
        ~student_df.set_index(["MA_SO_SV", "HOC_KY"]).index.isin(target_keys)
    ].copy()

    df = pd.concat([student_filtered, df], axis=0, ignore_index=True)
    df = df.sort_values(["MA_SO_SV", "SEMESTER_INDEX"])

    df["PTXT"] = df["PTXT"].replace({
        "5": "100",
        "3": "200",
        "1": "303"
    })

    def calculate_z_score(row):
        year = row["NAM_TUYENSINH"]
        score = row["DIEM_TRUNGTUYEN"]
        block = str(row["TOHOP_XT"]).upper().strip()

        if year not in EXAM_STATS_DETAILED:
            return np.nan

        year_stats = EXAM_STATS_DETAILED[year]
        if block in year_stats:
            stats = year_stats[block]
        else:
            stats = year_stats["OTHER"]

        mean_val = stats["mean"]
        std_val = stats["std"]

        if std_val < 0.1:
            std_val = 1.0

        return (score - mean_val) / std_val

    df["Z_SCORE"] = df.apply(calculate_z_score, axis=1)

    if "TC_HOANTHANH" in df.columns:
        df["FAIL_CREDITS"] = df["TC_DANGKY"] - df["TC_HOANTHANH"]
    else:
        df["FAIL_CREDITS"] = 0

    df["SCORE_GAP"] = df["DIEM_TRUNGTUYEN"] - df["DIEM_CHUAN"]
    df["GAP_RATIO"] = df["SCORE_GAP"] / (df["DIEM_CHUAN"] + 1.0)

    df["ENTRY_RANK"] = df.groupby("NAM_TUYENSINH")["DIEM_TRUNGTUYEN"].transform(
        lambda x: x.rank(pct=True, method='average')
    )
    df["BENCHMARK_TIER"] = df.groupby("NAM_TUYENSINH")["DIEM_CHUAN"].transform(
        lambda x: x.rank(pct=True)
    )

    grouped = df.groupby("MA_SO_SV")
    df["LAST_GPA"] = grouped["GPA"].shift(1)
    df["LAST_FAIL"] = grouped["FAIL_CREDITS"].shift(1)
    df["LAST_PASSED"] = grouped["TC_HOANTHANH"].shift(1)
    df["LAST_DANGKY"] = grouped["TC_DANGKY"].shift(1)
    df["LAST_PASS_RATIO"] = df["LAST_PASSED"] / df["LAST_DANGKY"]

    df["HIST_AVG_GPA"] = grouped["GPA"].transform(lambda x: x.shift(1).expanding().mean())
    df["TOTAL_EARNED"] = grouped["TC_HOANTHANH"].transform(lambda x: x.shift(1).fillna(0).cumsum())
    df["HIST_MAX_PASSED"] = grouped["TC_HOANTHANH"].transform(lambda x: x.shift(1).expanding().max())
    df["HIST_MAX_GPA"] = grouped["GPA"].transform(lambda x: x.shift(1).expanding().max())
    df["HIST_STD_GPA"] = grouped["GPA"].transform(lambda x: x.shift(1).expanding().std())
    df["OVERLOAD_VS_MAX"] = df["TC_DANGKY"] - df["HIST_MAX_PASSED"]

    window2 = grouped.rolling(window=2, min_periods=1, closed='left')
    df["R2_AVG_GPA"] = window2["GPA"].mean().reset_index(0, drop=True)
    df["R2_SUM_FAIL"] = window2["FAIL_CREDITS"].sum().reset_index(0, drop=True)
    df["R2_AVG_PASSED"] = window2["TC_HOANTHANH"].mean().reset_index(0, drop=True)
    df["R2_SUM_DANGKY"] = window2["TC_DANGKY"].sum().reset_index(0, drop=True)
    df["R2_SUM_PASSED"] = window2["TC_HOANTHANH"].sum().reset_index(0, drop=True)
    df["R2_PASS_RATE"] = df["R2_SUM_PASSED"] / df["R2_SUM_DANGKY"]
    df["PRESSURE_VS_R2"] = df["TC_DANGKY"] / df["R2_AVG_PASSED"]
    df["GPA_TREND_R2"] = df["R2_AVG_GPA"] - df["HIST_AVG_GPA"]
    df["FAIL_TREND_R2"] = df["LAST_FAIL"] - (df["R2_SUM_FAIL"] / 2)

    window3 = grouped.rolling(window=3, min_periods=1, closed='left')
    df["R3_AVG_GPA"] = window3["GPA"].mean().reset_index(0, drop=True)
    df["R3_SUM_FAIL"] = window3["FAIL_CREDITS"].sum().reset_index(0, drop=True)
    df["R3_AVG_PASSED"] = window3["TC_HOANTHANH"].mean().reset_index(0, drop=True)
    df["PRESSURE_VS_R3"] = df["TC_DANGKY"] / df["R3_AVG_PASSED"]
    df["OVERLOAD_R3"] = df["TC_DANGKY"] - df["R3_AVG_PASSED"]

    def parse_hoc_ky(hk_str):
        hk_str = str(hk_str)
        year_match = re.search(r"(\d{4})", hk_str)
        year = int(year_match.group(1)) if year_match else 2024
        return year

    df["YEAR_START"] = df["HOC_KY"].apply(parse_hoc_ky)
    df["SV_NAM_THU"] = df["YEAR_START"] - df["NAM_TUYENSINH"] + 1
    df.loc[df["SV_NAM_THU"] < 1, "SV_NAM_THU"] = 1

    fill_0 = [
        "LAST_FAIL", "R2_SUM_FAIL", "R3_SUM_FAIL", "FAIL_TREND_R2", "OVERLOAD_R3",
        "TOTAL_EARNED", "OVERLOAD_VS_MAX", "HIST_STD_GPA", "GPA_TREND_R2"
    ]
    df[fill_0] = df[fill_0].fillna(0.0)

    fill_1 = ["LAST_PASS_RATIO", "R2_PASS_RATE", "PRESSURE_VS_R2", "PRESSURE_VS_R3"]
    df[fill_1] = df[fill_1].fillna(1.0)

    fill_mean_gpa = ["LAST_GPA", "R2_AVG_GPA", "R3_AVG_GPA", "HIST_AVG_GPA", "HIST_MAX_GPA"]
    df[fill_mean_gpa] = df[fill_mean_gpa].fillna(academic_df["GPA"].mean())

    fill_15 = ["LAST_PASSED", "R2_AVG_PASSED", "R3_AVG_PASSED", "HIST_MAX_PASSED"]
    df[fill_15] = df[fill_15].fillna(15.0)

    df = df.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    final_df = pd.merge(
        input_df[["MA_SO_SV", "HOC_KY"]],
        df,
        on=["MA_SO_SV", "HOC_KY"],
        how="left"
    )

    return final_df

train_final = get_features(train_df_raw)
val_final = get_features(val_df_raw)
test_final = get_features(test_df_raw)

In [ ]:
print(train_final.columns)
print(val_final.columns)
print(test_final.columns)

Index(['MA_SO_SV', 'HOC_KY', 'CPA', 'GPA', 'TC_DANGKY', 'TC_HOANTHANH',
       'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN',
       'SEMESTER_INDEX', 'Z_SCORE', 'FAIL_CREDITS', 'SCORE_GAP', 'GAP_RATIO',
       'ENTRY_RANK', 'BENCHMARK_TIER', 'LAST_GPA', 'LAST_FAIL', 'LAST_PASSED',
       'LAST_DANGKY', 'LAST_PASS_RATIO', 'HIST_AVG_GPA', 'TOTAL_EARNED',
       'HIST_MAX_PASSED', 'HIST_MAX_GPA', 'HIST_STD_GPA', 'OVERLOAD_VS_MAX',
       'R2_AVG_GPA', 'R2_SUM_FAIL', 'R2_AVG_PASSED', 'R2_SUM_DANGKY',
       'R2_SUM_PASSED', 'R2_PASS_RATE', 'PRESSURE_VS_R2', 'GPA_TREND_R2',
       'FAIL_TREND_R2', 'R3_AVG_GPA', 'R3_SUM_FAIL', 'R3_AVG_PASSED',
       'PRESSURE_VS_R3', 'OVERLOAD_R3', 'YEAR_START', 'SV_NAM_THU'],
      dtype='object')
Index(['MA_SO_SV', 'HOC_KY', 'CPA', 'GPA', 'TC_DANGKY', 'TC_HOANTHANH',
       'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN',
       'SEMESTER_INDEX', 'Z_SCORE', 'FAIL_CREDITS', 'SCORE_GAP', 'GAP_RATIO',
       'ENTR

In [ ]:
target = "FAIL_CREDITS"
categorical_cols = ["PTXT", "TOHOP_XT"]

cbe = ce.CatBoostEncoder(
    cols=categorical_cols,
    handle_missing="return_nan"
)

train_final[categorical_cols] = cbe.fit_transform(
    train_final[categorical_cols],
    train_final[target]
)

val_final[categorical_cols] = cbe.transform(val_final[categorical_cols])
test_final[categorical_cols] = cbe.transform(test_final[categorical_cols])

print(f"Final Train shape: {train_final.shape}")
print(f"Final Val shape: {val_final.shape}")
print(f"Final Test shape: {test_final.shape}")

Final Train shape: (90122, 45)
Final Val shape: (15144, 45)
Final Test shape: (16502, 45)


In [ ]:
def split_by_year(df):
    df_fresh = df[df["SV_NAM_THU"] == 1].copy()
    df_senior = df[df["SV_NAM_THU"] > 1].copy()

    return df_fresh, df_senior

train_fresh, train_senior = split_by_year(train_final)
val_fresh, val_senior = split_by_year(val_final)
test_fresh, test_senior = split_by_year(test_final)

In [ ]:
feats_senior = [
    "TC_DANGKY", "SEMESTER_INDEX", "SV_NAM_THU",

    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",

    "R2_AVG_GPA", "R2_SUM_FAIL", "R2_PASS_RATE",
    "R3_AVG_GPA", "R3_SUM_FAIL",

    "PRESSURE_VS_R2", "PRESSURE_VS_R3", "OVERLOAD_R3",
    "FAIL_TREND_R2", "GPA_TREND_R2",

    "TOTAL_EARNED", "HIST_AVG_GPA",
    "HIST_MAX_PASSED",
    "HIST_MAX_GPA",
    "HIST_STD_GPA",
    "OVERLOAD_VS_MAX"
]

feats_fresh = [
    "TC_DANGKY", "SEMESTER_INDEX",
    "PTXT", "TOHOP_XT",

    "DIEM_TRUNGTUYEN", "DIEM_CHUAN",
    "SCORE_GAP", "ENTRY_RANK", "BENCHMARK_TIER",
    "Z_SCORE", "GAP_RATIO",

    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",
    "PRESSURE_VS_R2"
]

meta_cols = ["MA_SO_SV", "HOC_KY", "FAIL_CREDITS"]

def filter_cols(df, features):
    desired_cols = set(features + meta_cols)
    existing_cols = [c for c in df.columns if c in desired_cols]
    return df[existing_cols].copy()

train_fresh = filter_cols(train_fresh, feats_fresh)
val_fresh = filter_cols(val_fresh, feats_fresh)
test_fresh = filter_cols(test_fresh, feats_fresh)

train_senior = filter_cols(train_senior, feats_senior)
val_senior = filter_cols(val_senior, feats_senior)
test_senior = filter_cols(test_senior, feats_senior)

full_train_fresh = pd.concat([train_fresh, val_fresh], axis=0, ignore_index=True)
full_train_senior = pd.concat([train_senior, val_senior], axis=0, ignore_index=True)

In [ ]:
print(f"Train Fresher: {train_fresh.shape} | Train Senior: {train_senior.shape}")
print(f"Val Fresher: {val_fresh.shape} | Val Senior: {val_senior.shape}")
print(f"Test Fresher: {test_fresh.shape} | Test Senior: {test_senior.shape}")

Train Fresher: (24996, 18) | Train Senior: (65126, 25)
Val Fresher: (3504, 18) | Val Senior: (11640, 25)
Test Fresher: (4326, 18) | Test Senior: (12176, 25)


In [ ]:
FIXED_PARAMS = {
    "objective": "tweedie",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "random_state": RANDOM_SEED,
    "n_estimators": 4000,
    "device": DEVICE.type,
    "verbosity": -1,
    "boost_from_average": True
}

In [ ]:
def get_optuna_params(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "tweedie_variance_power": trial.suggest_float("tweedie_variance_power", 1.01, 1.99),

        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),

        "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 100, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 10.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),

        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 100.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 100.0, log=True),
        "path_smooth": trial.suggest_float("path_smooth", 0.0, 10.0),

        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),

        "max_bin": trial.suggest_categorical("max_bin", [255, 512]),
    }

    return params

def optimize_lgb(train_df, val_df, feats, n_trial, optim_type):
    base_score = train_df[target].mean()

    def objective_lgb(trial):
        params = {
            **FIXED_PARAMS,
            **get_optuna_params(trial),
        }

        model = LGBMRegressor(**params)
        callbacks = [
            early_stopping(stopping_rounds=100, verbose=False),
            log_evaluation(period=0)
        ]

        model.fit(
            train_df[feats],
            train_df[target],
            eval_set=[(val_df[feats], val_df[target])],
            eval_metric="rmse",
            callbacks=callbacks
        )

        best_iter = model.best_iteration_
        preds = model.predict(val_df[feats], num_iteration=best_iter)

        limit = val_df["TC_DANGKY"]
        preds = np.clip(preds, 0, limit)

        rmse = np.sqrt(mean_squared_error(val_df[target].values.ravel(), preds))
        return rmse

    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
    )

    with tqdm(total=n_trial, desc=f"LGBM {optim_type} Tuning") as pbar:
        def tqdm_callback(study, trial):
            pbar.update(1)
            pbar.set_postfix({"Best RMSE": f"{study.best_value:.4f}"})

        study.optimize(objective_lgb, n_trials=n_trial, callbacks=[tqdm_callback])

    best_params = {**FIXED_PARAMS, **study.best_params}
    best_rmse = study.best_value
    return best_params, best_rmse

In [ ]:
N_TRIALS = 300

best_fresh_params, best_fresh_rmse = optimize_lgb(
    train_fresh, val_fresh, feats_fresh, N_TRIALS, "Fresher"
)

best_senior_params, best_senior_rmse = optimize_lgb(
    train_senior, val_senior, feats_senior, N_TRIALS, "Senior"
)

LGBM Fresher Tuning:   0%|          | 0/300 [00:00<?, ?it/s]

LGBM Senior Tuning:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
print(f"Best Fresher RMSE: {best_fresh_rmse:.4f}")
print(f"Best Senior RMSE: {best_senior_rmse:.4f}")

print(f"Best Fresher Params: {best_fresh_params}")
print(f"Best Senior Params: {best_senior_params}")

Best Fresher RMSE: 3.9040
Best Senior RMSE: 3.6730
Best Fresher Params: {'objective': 'tweedie', 'metric': 'rmse', 'boosting_type': 'gbdt', 'random_state': 42, 'n_estimators': 4000, 'device': 'cpu', 'verbosity': -1, 'boost_from_average': True, 'learning_rate': 0.024656903270396133, 'tweedie_variance_power': 1.8980766285207884, 'num_leaves': 284, 'max_depth': 12, 'min_child_weight': 0.007600591613729644, 'min_split_gain': 0.0033225863743921724, 'min_child_samples': 41, 'reg_alpha': 0.1968358854659421, 'reg_lambda': 1.5442085421741997e-08, 'path_smooth': 6.111762711703784, 'subsample': 0.916983905876348, 'subsample_freq': 5, 'colsample_bytree': 0.9684893062778903, 'extra_trees': True, 'max_bin': 512}
Best Senior Params: {'objective': 'tweedie', 'metric': 'rmse', 'boosting_type': 'gbdt', 'random_state': 42, 'n_estimators': 4000, 'device': 'cpu', 'verbosity': -1, 'boost_from_average': True, 'learning_rate': 0.0020927804628431464, 'tweedie_variance_power': 1.0750084049393747, 'num_leaves': 

In [ ]:
def train_lgb(params, train_df, val_df, feats, model_type):
    print(f"Training {model_type} Model...")

    model = LGBMRegressor(**params)
    callbacks = [
        early_stopping(stopping_rounds=100),
        log_evaluation(period=100)
    ]

    model.fit(
        train_df[feats],
        train_df[target],
        eval_set=[(val_df[feats], val_df[target])],
        eval_metric="rmse",
        callbacks=callbacks
    )

    best_iter = model.best_iteration_
    preds = model.predict(val_df[feats], num_iteration=best_iter)

    limit = val_df["TC_DANGKY"]
    preds = np.clip(preds, 0, limit)

    return best_iter, preds

best_iter_fresh, preds_fresh = train_lgb(
    best_fresh_params, train_fresh, val_fresh, feats_fresh, "Fresher"
)

best_iter_senior, preds_senior = train_lgb(
    best_senior_params, train_senior, val_senior, feats_senior, "Senior"
)

Training Fresher Model...
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 4.13597
[200]	valid_0's rmse: 3.93
[300]	valid_0's rmse: 3.91489
[400]	valid_0's rmse: 3.92554
Early stopping, best iteration is:
[310]	valid_0's rmse: 3.91357
Training Senior Model...
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 4.88809
[200]	valid_0's rmse: 4.56056
[300]	valid_0's rmse: 4.31658
[400]	valid_0's rmse: 4.13554
[500]	valid_0's rmse: 4.00461
[600]	valid_0's rmse: 3.9105
[700]	valid_0's rmse: 3.84372
[800]	valid_0's rmse: 3.79617
[900]	valid_0's rmse: 3.76245
[1000]	valid_0's rmse: 3.73804
[1100]	valid_0's rmse: 3.72075
[1200]	valid_0's rmse: 3.70937
[1300]	valid_0's rmse: 3.70042
[1400]	valid_0's rmse: 3.69398
[1500]	valid_0's rmse: 3.68923
[1600]	valid_0's rmse: 3.68578
[1700]	valid_0's rmse: 3.68299
[1800]	valid_0's rmse: 3.68093
[1900]	valid_0's rmse: 3.67944
[2000]	valid_0's rmse: 3.67822
[2100]	valid_0's rmse: 3.67731
[2

In [ ]:
val_targets = np.concatenate([val_fresh[target].values, val_senior[target].values])
val_preds = np.concatenate([preds_fresh, preds_senior])

def calculate_wmape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    if np.sum(y_true) == 0:
        return 0
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
mse = mean_squared_error(val_targets, val_preds)
r2 = r2_score(val_targets, val_preds)
wmape = calculate_wmape(val_targets, val_preds)

print(f"RMSE : {rmse:.4f}")
print(f"MSE : {mse:.4f}")
print(f"R^2 : {r2:.4f}")
print(f"wMAPE : {wmape:.4f}")

RMSE : 3.7277
MSE : 13.8958
R^2 : 0.5097
wMAPE : 0.6266


In [ ]:
def test_lgb(params, best_iter, full_train_df, train_df, test_df, feats, model_type):
    print(f"Testing {model_type} Model...")

    scale_ratio = len(full_train_df) / len(train_df)
    final_params = params.copy()
    final_params["n_estimators"] = int(best_iter * scale_ratio)

    if "early_stopping_rounds" in final_params:
        del final_params["early_stopping_rounds"]

    model = LGBMRegressor(**final_params)
    model.fit(
        full_train_df[feats],
        full_train_df[target],
        eval_metric="rmse"
    )

    model.booster_.save_model(f"{WORK_DIR}/lgbm_{model_type.lower()}.txt")

    preds = model.predict(test_df[feats])
    limit = test_df["TC_DANGKY"]
    preds = np.clip(preds, 0, limit)
    preds = limit - preds
    return preds

preds_fresh = test_lgb(
    best_fresh_params, best_iter_fresh,
    full_train_fresh, train_fresh, test_fresh, feats_fresh, "Fresher"
)

preds_senior = test_lgb(
    best_senior_params, best_iter_senior,
    full_train_senior, train_senior, test_senior, feats_senior, "Senior"
)

Testing Fresher Model...
Testing Senior Model...


In [ ]:
sub_fresh_df = pd.DataFrame({
    "MA_SO_SV": test_fresh["MA_SO_SV"],
    "PRED_TC_HOANTHANH": preds_fresh
})

sub_senior_df = pd.DataFrame({
    "MA_SO_SV": test_senior["MA_SO_SV"],
    "PRED_TC_HOANTHANH": preds_senior
})

preds_df = pd.concat([sub_fresh_df, sub_senior_df], axis=0)

submission = pd.merge(
    test_df_raw[["MA_SO_SV"]],
    preds_df,
    on=["MA_SO_SV"],
    how="left"
)

submission.to_csv(SUBMISSION_CSV, index=False)

In [ ]:
print(submission)

           MA_SO_SV  PRED_TC_HOANTHANH
0      481436e2064d           1.550859
1      6c8a97d22131           2.527446
2      e87f62beabbb           4.937049
3      438aff5ef524           0.000000
4      ad172a9b0722          14.929568
...             ...                ...
16497  9e803a0d26f0          41.495951
16498  dbc819721795          51.422594
16499  9e1c8deafb70          42.908483
16500  ffecfc70f83a          45.555785
16501  dc7b37953745          51.881950

[16502 rows x 2 columns]
